In [2]:
%pip install pandas
%pip install openpyxl
import pandas as pd

data_source = 'local'

prilohy_path = {
    'A': '../data/prilohaA-napocet.xlsx',
    'B': '../data/prilohaB-crepc.xlsx',
    'C': '../data/prilohaC-monografie.xlsx',
    'D': '../data/prilohaD-granty.xlsx',
} if data_source == 'local' else {
    'A': 'https://www.minedu.sk/data/att/25844.xlsx',
    'B': 'https://www.minedu.sk/data/att/25563.xlsx',
    'C': 'https://www.minedu.sk/data/att/25537.xlsx',
    'D': 'https://www.minedu.sk/data/att/25538.xlsx',
}
prilohaA = pd.read_excel(prilohy_path['A'], sheet_name = None, header=None)
print(prilohaA.keys())
# ['E1_alokácia', 'E2_zamestnanci_2021', 'E3_oblasti', 'E4a_M1_prirodne', 'E4b_M2_technicke', 'E4c_M3_lekarske', 'E4d_M4_polno_les_vet', 'E4e_M5_spolocenske', 'E4f_M6_humanitne', 'E4g_M6_umenie']
prilohaB = pd.read_excel(prilohy_path['B'], sheet_name = None)
print(prilohaB.keys())
# ['2021 - 3']
prilohaC = pd.read_excel(prilohy_path['C'], sheet_name = None)
print(prilohaC.keys())
# ['crepc 2020', 'crepc 2021', 'crepc 2022', 'crepc 2022-2', 'CVTI_NORDIC', 'Nordic_CVTI 2020', 'Nordic_CVTI 2021', 'vystupy 2020', 'vystupy 2021', 'Hárok1']
prilohaD = pd.read_excel(prilohy_path['D'], sheet_name = None)
print(prilohaD.keys())
# ['T3 - výsk. zahr. grant. schémy', 'old', 'oblasti výskumu', 'VŠ', 'Odbory VaT']



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
dict_keys(['E1_alokácia', 'E2_zamestnanci_2021', 'E3_oblasti', 'E4a_M1_prirodne', 'E4b_M2_technicke', 'E4c_M3_lekarske', 'E4d_M4_polno_les_vet', 'E4e_M5_spolocenske', 'E4f_M6_humanitne', 'E4g_M6_umenie'])
dict_keys(['2021 - 3'])
dict_keys(['crepc 2020', 'crepc 2021', 'crepc 2022', 'crepc 2022-2', 'CVTI_NORDIC', 'Nordic_CVTI 2020', 'Nordic_CVTI 2021', 'vystupy 2020', 'vystupy 2021', 'Hárok1'])
dict_keys(['T3 - výsk. zahr. grant. schémy', 'old', 'oblasti výskumu', 'VŠ', 'Odbory VaT'])


/home/viki/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/viki/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/viki/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/viki/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [17]:
id2oblasti = prilohaA['E3_oblasti'].copy()
id2oblasti.index = id2oblasti[0]
id2oblasti = id2oblasti.to_dict()[2]

id2oblasti.update({key: value for key, value in zip([
'PRÍRODNÉ VEDY',
'TECHNICKÉ VEDY', 
'LEKÁRSKE VEDY',
'PÔDOHOSPODÁRSKE VEDY',
'SPOLOČENSKÉ VEDY',
'HUMANITNÉ VEDY',
], [f'M{i}' for i in range(1,7)])})
id2oblasti

{91: 'M1',
 92: 'M1',
 120: 'M1',
 130: 'M1',
 240: 'M1',
 50: 'M2',
 100: 'M2',
 110: 'M2',
 140: 'M2',
 150: 'M2',
 160: 'M2',
 170: 'M2',
 180: 'M3',
 190: 'M4',
 200: 'M4',
 10: 'M5',
 60: 'M5',
 70: 'M5',
 80: 'M5',
 210: 'M5',
 220: 'M2',
 230: 'M5',
 20: 'M6',
 30: 'M6',
 40: 'M6',
 'PRÍRODNÉ VEDY': 'M1',
 'TECHNICKÉ VEDY': 'M2',
 'LEKÁRSKE VEDY': 'M3',
 'PÔDOHOSPODÁRSKE VEDY': 'M4',
 'SPOLOČENSKÉ VEDY': 'M5',
 'HUMANITNÉ VEDY': 'M6'}

a) excelentná publikačná činnosť podľa bodu 4, celková váha 60%.

a) V oblastiach **M1**: prírodné vedy, **M2**: technické vedy, **M3**: lekárske vedy, **M4**: poľnohospodárske, lesnícke a veterinárske vedy sa zohľadňujú **váhou 100%** indexované publikácie (WoS, Scopus), teda kategórie **ADC, ADD, ADM, ADN**. Zoznam sa nachádza **v prílohe B**. Všetky výstupy sa **váhujú** podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný **výstup je 1,00**).

b) V oblasti **M5**: spoločenské vedy sa zohľadňujú **váhou 75%** indexované publikácie **(WoS, Scopus)**, teda kategórie **ADC, ADD, ADM, ADN**. Tieto výstupy sa bonifikujú v súlade s metodikou rozpisu **(prvý kvartil JCR je zohľadnený váhou 6, druhý kvartil váhou 4, tretí kvartil váhou 1 a štvrtý kvartil váhou 0,5)**. Zoznam sa nachádza **v prílohe B**. **Váhou 25%** sa zohľadňujú monografie (**AAA, AAB**). Tieto výstupy sa **bonifikujú** tak, že vydavateľstvá zaradené v databáze Nordic List (https://kanalregister.hkdir.no/publiseringskanaler/Forside.action?request_locale=en) v kategórii 2 (prestížne vedecké vydavateľstvo) sú bonifikované váhou 6, vydavateľstvá zaradené v databáze Nordic List v kategórii 1 (vedecké vydavateľstvo) váhou 4, vydavateľstvá zaradené v databáze CVTI (zoznam zahraničných vydavateľstiev: http://cms.crepc.sk/Data/Sites/1/pdf/zoznam-vydavatelstva/zoznam-vydavatelstva-01-2020.pdf) sú zohľadnené váhou 1 a všetky ostatné monografie sú zohľadnené váhou 0,5). Zoznam aj so zaradením sa nachádza v prílohe C. Všetky výstupy sa váhujú podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný výstup je 1,00).

c) V oblasti M6a: humanitné vedy sa zohľadňujú váhou 60% indexované publikácie (WoS, Scopus), teda kategórie ADC, ADD, ADM, ADN. Tieto výstupy sa bonifikujú v súlade s metodikou rozpisu (prvý kvartil JCR je zohľadnený váhou 6, druhý kvartil váhou 4, tretí kvartil váhou 1 a štvrtý kvartil váhou 0,5). Zoznam sa nachádza v prílohe B. Váhou 40% sa zohľadňujú monografie (AAA, AAB). Tieto výstupy sa bonifikujú tak, že vydavateľstvá zaradené v databáze Nordic List (https://kanalregister.hkdir.no/publiseringskanaler/Forside.action?request_locale=en) v kategórii 2 (prestížne vedecké vydavateľstvo) sú bonifikované váhou 6, vydavateľstvá zaradené v databáze Nordic List v kategórii 1 (vedecké vydavateľstvo) váhou 4, vydavateľstvá zaradené v databáze CVTI (zoznam zahraničných vydavateľstiev: http://cms.crepc.sk/Data/Sites/1/pdf/zoznam-vydavatelstva/zoznam-vydavatelstva-01-2020.pdf) sú zohľadnené váhou 1 a všetky ostatné monografie sú zohľadnené váhou 0,5). Zoznam aj so zaradením sa nachádza v prílohe C. Všetky výstupy sa váhujú podľa pracovísk tak, aby každý výstup bol zarátaný iba raz (súčet všetkých váh za daný výstup je 1,00).

d) v oblasti M6b umenie sa zohľadňujú váhou 100% umelecké výstupy v kategóriách ZZZ, ZZY, ZYZ, ZYY podľa váh platných pre rozpis dotácie 2022. Údaje sa nachádzajú v rozpise dotácie na rok 2022, hárok E4g_M6_umenie. Osobitne sa zohľadňujú oblasti performatívne umenie a vizuálne umenie.

In [4]:
# prilohy B, C
epc_codes_ADx = ['ADC', 'ADD', 'ADM', 'ADN']
epc_codes_AAx = ['AAA', 'AAB']
epc_codes_Zxx = ['ZZZ', 'ZZY', 'ZYZ', 'ZYY']

epc_codes_M1_M4 = epc_codes_ADx
epc_codes_M5 = epc_codes_ADx + epc_codes_AAx # TODO bonifikacia
epc_codes_M6a = epc_codes_ADx + epc_codes_AAx # TODO bonifikacia
epc_codes_M6b = epc_codes_Zxx # E4g_M6_umenie

publikacie_ADx = prilohaB['2021 - 3'].copy()
assert publikacie_ADx['EPC_KOD'].isin(epc_codes_ADx).all()

publikacie_AAx_2020 = prilohaC['crepc 2020'].copy()
assert publikacie_AAx_2020['EPC_KOD'].isin(epc_codes_AAx).all()

publikacie_AAx_2021 = prilohaC['crepc 2021'].copy()
assert publikacie_AAx_2021['EPC_KOD'].isin(epc_codes_AAx).all()

# TODO filter out only 2020-2021
casovy_ramec = [2020, 2021]

vysledne_pocty_fakult_v_oblastiach = {
    'M1': 24,
    'M2': 40,
    'M3': 11,
    'M4': 10,
    'M5': 70,
    'M6': 47,
}

In [29]:
# inverzny dict
# vyfiltrovat iba riadky co maju spravny kod
# 

stlpce = ['VS_NAZOV', 'FAKULTA_NAZOV', 'Výskyt', 'Váha', 'JCR_bonus']
melt_stlpce = [f"OBLAST_VYSKUMU_KOD{i}" for i in ["", 2, 3, 4, 5]]
publikacie_ADx_melted = (
    publikacie_ADx.loc[:, stlpce + melt_stlpce]
        .melt(id_vars=stlpce, value_name='kod')
        .drop(columns = ['variable'])
        .dropna(subset = ['kod'])
)
publikacie_ADx_melted['oblast'] = publikacie_ADx_melted['kod'].map(id2oblasti)
publikacie_ADx_melted


,VS_NAZOV,FAKULTA_NAZOV,Výskyt,Váha,JCR_bonus,kod,oblast
0,Prešovská univerzita v Prešove,Gréckokatolícka teologická fakulta,1,1.000000,0.5,20.0,M6
1,Katolícka univerzita v Ružomberku,Teologická fakulta,1,1.000000,0.5,20.0,M6
2,Univerzita Konštantína Filozofa v Nitre,Filozofická fakulta,1,1.000000,0.5,20.0,M6
3,Katolícka univerzita v Ružomberku,Teologická fakulta,1,1.000000,0.5,20.0,M6
4,Univerzita Konštantína Filozofa v Nitre,Filozofická fakulta,3,0.333333,0.5,20.0,M6
...,...,...,...,...,...,...,...
74959,Technická univerzita v Košiciach,"Fakulta baníctva, ekológie, riadenia a geotech...",4,0.250000,4.0,170.0,M2
75460,Akadémia Policajného zboru v Bratislave,Pracoviská APZ,2,0.500000,0.5,80.0,M5
75461,Univerzita Mateja Bela v Banskej Bystrici,Fakulta politických vied a medzinárodných vzťahov,2,0.500000,0.5,80.0,M5
86523,Univerzita Konštantína Filozofa v Nitre,Fakulta prírodných vied a informatiky,2,0.500000,0.5,240.0,M1


In [37]:
# get all distinct (VS_NAZOV, FAKULTA_NAZOV) in publikacie_ADx which have oblast M1
index_columns = ['VS_NAZOV', 'FAKULTA_NAZOV']
fakulty_v_oblasti = {
    oblast: publikacie_ADx_melted[publikacie_ADx_melted['oblast'] == oblast]
        .loc[:, index_columns]
        .drop_duplicates()
        .set_index(index_columns)
    for oblast in set(id2oblasti.values())
}

In [38]:
for value in fakulty_v_oblasti.values():
    value['sum_ADx'] = 0.0

for i, riadok in publikacie_ADx_melted.iterrows():
    pracovisko = tuple(riadok[key] for key in index_columns)
    fakulty_v_oblasti[riadok['oblast']].loc[pracovisko]['sum_ADx'] += riadok['Váha']


fakulty_v_oblasti['M1'].sort_values(by='sum_ADx', ascending=False)


sum_ADx
VS_NAZOV                                      FAKULTA_NAZOV                                             
Univerzita Komenského v Bratislave            Prírodovedecká fakulta UK                       942.595635
                                              Fakulta matematiky, fyziky a informatiky UK     589.422222
Univerzita Pavla Jozefa Šafárika v Košiciach  Prírodovedecká fakulta                          497.566667
Slovenská technická univerzita v Bratislave   Fakulta chemickej a potravinárskej technológie  365.716667
Univerzita Konštantína Filozofa v Nitre       Fakulta prírodných vied a informatiky           117.733333
...                                                                                                  ...
Univerzita Komenského v Bratislave            Pedagogická fakulta UK                            0.333333
                                              Filozofická fakulta UK                            0.333333
Prešovská univerzita v Prešove                Centrum jazykov a kultúr národnostných menšín     0.250000
Technická univerzita v Košiciach              Rektorát                                          0.250000
Slovenská poľnohospodárska univerzita v Nitre Botanická záhrada                                 0.250000

[96 rows x 1 columns]

5. Excelentný výkon pri získavaní výskumných grantov sa určuje nasledovne: pre všetky oblasti okrem M6b (umenie) sa zohľadňuje objem grantových prostriedkov, získaných súťažným spôsobom v kategórii zahraničné výskumné granty. Zoznam sa nachádza v prílohe D. Zahraničné výskumné granty sa priraďujú iba tým pracoviskám, v ktorých vysoké školy uviedli príslušných zamestnancov v danej oblasti.



In [48]:
granty = prilohaD['T3 - výsk. zahr. grant. schémy'].copy()
# granty = granty[granty['Rok'].isin(casovy_ramec)]

vyhra_key = '''Výška finančných prostriedkov v kategórii BV prijatých vysokou školou na jej účet v období od 1.1. do 31.12.2021
(uviesť v eurách v celých jednotkách)'''
granty.rename(columns = {
    vyhra_key: 'Výhra',
    'Vysoká škola': 'VS_NAZOV',
    'Názov fakulty': 'FAKULTA_NAZOV',
    }, inplace = True)

relevant_columns = index_columns + ['SKUPINA ODBOROV VEDY A TECHNIKY', 'Rok', 'Výhra']
granty = granty.loc[:, relevant_columns].dropna(subset = ['Výhra'])

granty['oblast'] = granty['SKUPINA ODBOROV VEDY A TECHNIKY'].map(id2oblasti)
granty

,VS_NAZOV,FAKULTA_NAZOV,SKUPINA ODBOROV VEDY A TECHNIKY,Rok,Výhra,oblast
0,STU Bratislava,Fakulta chemickej a potravinárskej technológie...,TECHNICKÉ VEDY,2021,22025.58,M2
1,STU Bratislava,Fakulta chemickej a potravinárskej technológie...,PRÍRODNÉ VEDY,2020,8045.52,M1
2,ŽU Žilina,Fakulta prevádzky a ekonomiky dopravy a spojov ŽU,SPOLOČENSKÉ VEDY,2021,11324.00,M5
3,TU Košice,Ekonomická fakulta TUKE,SPOLOČENSKÉ VEDY,2020,9635.00,M5
4,TU Košice,"Fakulta baníctva, ekológie, riadenia a geotech...",TECHNICKÉ VEDY,2021,84860.34,M2
...,...,...,...,...,...,...
487,STU Bratislava,Materiálovotechnologická fakulta STU v Trnave,TECHNICKÉ VEDY,2020,3063.45,M2
488,STU Bratislava,Materiálovotechnologická fakulta STU v Trnave,TECHNICKÉ VEDY,2020,33138.00,M2
489,STU Bratislava,Ústav manažmentu STU,SPOLOČENSKÉ VEDY,2020,40438.79,M5
490,STU Bratislava,Ústav manažmentu STU,SPOLOČENSKÉ VEDY,2020,110344.48,M5


In [50]:
# TODO fakulty_v_oblasti do novej premennej a merge so starou
fakulty_v_oblasti = {
    oblast: granty[granty['oblast'] == oblast]
        .loc[:, index_columns]
        .drop_duplicates()
        .set_index(index_columns)
    for oblast in set(id2oblasti.values())
}

for value in fakulty_v_oblasti.values():
    value['sum_granty'] = 0.0

for i, riadok in granty.iterrows():
    pracovisko = tuple(riadok[key] for key in index_columns)
    fakulty_v_oblasti[riadok['oblast']].loc[pracovisko]['sum_granty'] += riadok['Výhra']


fakulty_v_oblasti['M1'].sort_values(by='sum_granty', ascending=False)

sum_granty
VS_NAZOV            FAKULTA_NAZOV                                                 
TUAD Trenčín        NaN                                                 1928697.00
UK Bratislava       Fakulta matematiky, fyziky a informatiky UK         1431890.55
                    Prírodovedecká fakulta UK                           1331560.16
UPJŠ Košice         NaN                                                  323296.88
                    Prírodovedecká fakulta UPJŠ                          275499.31
STU Bratislava      Fakulta elektrotechniky a informatiky STU            271309.02
TU Zvolen           Drevárska fakulta TUZVO                              262117.00
UKF Nitra           Fakulta prírodných vied UKF                          245204.00
STU Bratislava      Fakulta chemickej a potravinárskej technológie STU    92845.71
SPU Nitra           Fakulta biotechnológie a potravinárstva SPU           33400.00
ŽU Žilina           Fakulta elektrotechniky a informačných technoló...    30404.00
UMB Banská Bystrica Fakulta prírodných vied UMB                           24831.00
TU Košice           Fakulta elektrotechniky a informatiky TUKE            22000.00
STU Bratislava      Stavebná fakulta STU                                  20320.00
PU Prešov           Fakulta humanitných a prírodných vied PU              20000.00
ŽU Žilina           Výskumné centrum ŽU                                   18954.00
STU Bratislava      Materiálovotechnologická fakulta STU v Trnave         15000.00
UCM Trnava          Fakulta prírodných vied UCM                            9629.00
TVU Trnava          Pedagogická fakulta TVU                                6807.00
UK Bratislava       Farmaceutická fakulta UK                               6360.00